In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext
import pandas as pd
import datetime
import import_ipynb
import utils_cdt as cdt
import pandas as pd

importing Jupyter notebook from utils_cdt.ipynb


In [2]:
data_vencimento = 10

# Prioridade 1 - SaldoAtualFinal

In [3]:
infoDataBases = {"ip":"AZ2D-MASKDB-221.DEVCDT.COM.BR","base":"Agiplan"}

In [4]:
consulta = cdt.resultQuery("mdl_historicoscorrentes.sql").format(dia=data_vencimento)
base = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="mdl_historicoscorrentes_"+infoDataBases["base"],spark=False,objSpark=None)

In [5]:
base['DataVencimentoPadrao'].unique()

array(['10/01/2020', '10/02/2020', '10/03/2020', '10/04/2020'],
      dtype=object)

In [6]:
base = base.fillna(0)
base['dtvencimento'] = pd.to_datetime(base.DataVencimentoPadrao, format="%d/%m/%Y")

base = base.loc[base.dtvencimento.dt.day == data_vencimento]

mylist = ["SaldoExtratoAnterior","FinanciavelExtrato","SaldoAtualFinal","CreditosNacionais","DebitosNacionais","SaquesNacionais","Multa","FinanciavelAtualFinal","NaoFinanciavelAtualFinal","BaseMultaCobrada"]
zeros = base[base[mylist].eq(0).all(1)]

final = base.drop(zeros.index, axis=0)

treino = final.drop(final.loc[base.dtvencimento.dt.month == 4].loc[base.dtvencimento.dt.year == 2020].index,axis = 0)
teste = final.loc[base.dtvencimento.dt.month == 4].loc[base.dtvencimento.dt.year == 2020]

## Pré-Processamento - SaldoAtualFinal

In [7]:
from sklearn.preprocessing import StandardScaler
import time
start_time = time.time()

X = treino.drop(['SaldoAtualFinal','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
Y = treino['SaldoAtualFinal'].to_frame()

scalerX = StandardScaler()
scalerX.fit(X)
Xss = scalerX.transform(X)

scalerY = StandardScaler()
scalerY.fit(Y)
Yss = scalerY.transform(Y)

print(str(time.time() - start_time))

0.2700016498565674


## Decision Tree - SaldoAtualFinal

In [8]:
from sklearn import tree
dt = tree.DecisionTreeRegressor()
dt = dt.fit(Xss, Yss)
mdl_saldoatualfinal = dt

## Validate - SaldoAtualFinal

In [9]:
realY = teste['SaldoAtualFinal'].values
realX =  teste.drop(['SaldoAtualFinal','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
realXS = scalerX.transform(realX)

preditoYS = dt.predict(realXS)
preditoY = scalerY.inverse_transform(preditoYS)

df_dict = {'Real':realY.astype(int),'Previsto': preditoY.astype(int)}

rp = pd.DataFrame(df_dict)

a = preditoY.astype(int).reshape(preditoY.astype(int).shape[0],1)
b = realY.astype(int).reshape(realY.astype(int).shape[0],1)

a = preditoY.reshape(preditoY.shape[0],1)
df_final = cdt.transformDataSet(a, b)
cdt.getresultdataset(df_final, "DecisionTreeRegressor")

SE - Sum of Error: 
[DecisionTreeRegressor] - Total Real:  235351390.00
[DecisionTreeRegressor] - Total Prev:  235348882.81
[DecisionTreeRegressor] - SE Final:  -2507.50
[DecisionTreeRegressor] - Total Porcentagem de diferença do previsto x real:  0.00%
----------------------------------------------------------
ME - Mean Error:  
[DecisionTreeRegressor] - Total Registros: 243479
[DecisionTreeRegressor] - Total Diff : -2507
[DecisionTreeRegressor] - ME Final : -0.010299
----------------------------------------------------------
MAE - Mean Absolute Error:  
[DecisionTreeRegressor] - Total Registros: 243479
[DecisionTreeRegressor] - Total Absolute Diff : 1112526
[DecisionTreeRegressor] - ME Final : 4.569287
----------------------------------------------------------
MPE - Mean Percentage Error:  
[DecisionTreeRegressor] - Total Registros: 243479
[DecisionTreeRegressor] - Total percent Diff: 64111
[DecisionTreeRegressor] - MPE Final : 0.26%
--------------------------------------------------

# Prioridade 2 - CreditosNacionais

In [10]:
import pandas as pd
consulta = cdt.resultQuery("mdl_historicoscorrentes.sql").format(dia=data_vencimento)
base = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="mdl_historicoscorrentes_"+infoDataBases["base"],spark=False,objSpark=None)
base['dtvencimento'] = pd.to_datetime(base.DataVencimentoPadrao, format="%d/%m/%Y")

base = base.loc[base.dtvencimento.dt.day == data_vencimento]

mylist = ["SaldoExtratoAnterior","FinanciavelExtrato","SaldoAtualFinal","CreditosNacionais","DebitosNacionais","SaquesNacionais","Multa","FinanciavelAtualFinal","NaoFinanciavelAtualFinal","BaseMultaCobrada"]
zeros = base[base[mylist].eq(0).all(1)]

final = base.drop(zeros.index, axis=0)

treino = final.drop(final.loc[base.dtvencimento.dt.month == 4].loc[base.dtvencimento.dt.year == 2020].index,axis = 0)
teste = final.loc[base.dtvencimento.dt.month == 4].loc[base.dtvencimento.dt.year == 2020]

## Pré-Processamento - CreditosNacionais

In [11]:
from sklearn.preprocessing import StandardScaler
import time
start_time = time.time()

X = treino.drop(['CreditosNacionais','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
Y = treino['CreditosNacionais'].to_frame()

scalerX = StandardScaler()
scalerX.fit(X)
Xss = scalerX.transform(X)

scalerY = StandardScaler()
scalerY.fit(Y)
Yss = scalerY.transform(Y)

print(str(time.time() - start_time))

0.3139984607696533


## Decision Tree - CréditosNacionais

In [12]:
from sklearn import tree

dt = tree.DecisionTreeRegressor()
dt = dt.fit(Xss, Yss)
mdl_creditosnacionais = dt

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## Validate - CréditosNacionais

In [13]:
realY = teste['CreditosNacionais'].values
realX =  teste.drop(['CreditosNacionais','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
realXS = scalerX.transform(realX)

preditoYS = dt.predict(realXS)
preditoY = scalerY.inverse_transform(preditoYS)

df_dict = {'Real':realY.astype(int),'Previsto': preditoY.astype(int)}

rp = pd.DataFrame(df_dict)

a = preditoY.astype(int).reshape(preditoY.astype(int).shape[0],1)
b = realY.astype(int).reshape(realY.astype(int).shape[0],1)

a = preditoY.reshape(preditoY.shape[0],1)
df_final = cdt.transformDataSet(a, b)
cdt.getresultdataset(df_final, "DecisionTreeRegressor")

NotFittedError: This DecisionTreeRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# Prioridade 3 - DebitosNacionais

In [14]:
import pandas as pd
consulta = cdt.resultQuery("mdl_historicoscorrentes.sql").format(dia=data_vencimento)
base = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="mdl_historicoscorrentes_"+infoDataBases["base"],spark=False,objSpark=None)
base = base.fillna(0)
base['dtvencimento'] = pd.to_datetime(base.DataVencimentoPadrao, format="%d/%m/%Y")

base = base.loc[base.dtvencimento.dt.day == data_vencimento]

mylist = ["SaldoExtratoAnterior","FinanciavelExtrato","SaldoAtualFinal","CreditosNacionais","DebitosNacionais","SaquesNacionais","Multa","FinanciavelAtualFinal","NaoFinanciavelAtualFinal","BaseMultaCobrada"]
zeros = base[base[mylist].eq(0).all(1)]

final = base.drop(zeros.index, axis=0)

treino = final.drop(final.loc[base.dtvencimento.dt.month == 4].loc[base.dtvencimento.dt.year == 2020].index,axis = 0)
teste = final.loc[base.dtvencimento.dt.month == 4].loc[base.dtvencimento.dt.year == 2020]

## Pré-Processamento - DebitosNacionais

In [15]:
from sklearn.preprocessing import StandardScaler
import time
start_time = time.time()

X = treino.drop(['DebitosNacionais','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
Y = treino['DebitosNacionais'].to_frame()

scalerX = StandardScaler()
scalerX.fit(X)
Xss = scalerX.transform(X)

scalerY = StandardScaler()
scalerY.fit(Y)
Yss = scalerY.transform(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xss, Yss, test_size = 0.3,random_state = 0)

print(str(time.time() - start_time))

0.44899797439575195


## Decision Tree - DebitosNacionais

In [16]:
from sklearn import tree
dt = tree.DecisionTreeRegressor()
dt = dt.fit(X_train, y_train)
mdl_debitosnacionais = dt

## Validate - DebitosNacionais

In [17]:
realY = teste['DebitosNacionais'].values
realX =  teste.drop(['DebitosNacionais','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
realXS = scalerX.transform(realX)

preditoYS = dt.predict(realXS)
preditoY = scalerY.inverse_transform(preditoYS)

df_dict = {'Real':realY.astype(int),'Previsto': preditoY.astype(int)}

rp = pd.DataFrame(df_dict)

a = preditoY.astype(int).reshape(preditoY.astype(int).shape[0],1)
b = realY.astype(int).reshape(realY.astype(int).shape[0],1)

a = preditoY.reshape(preditoY.shape[0],1)
df_final = cdt.transformDataSet(a, b)
cdt.getresultdataset(df_final, "DecisionTreeRegressor")

SE - Sum of Error: 
[DecisionTreeRegressor] - Total Real:  20213830.00
[DecisionTreeRegressor] - Total Prev:  20235976.98
[DecisionTreeRegressor] - SE Final:  22147.75
[DecisionTreeRegressor] - Total Porcentagem de diferença do previsto x real:  0.11%
----------------------------------------------------------
ME - Mean Error:  
[DecisionTreeRegressor] - Total Registros: 242608
[DecisionTreeRegressor] - Total Diff : 22148
[DecisionTreeRegressor] - ME Final : 0.091290
----------------------------------------------------------
MAE - Mean Absolute Error:  
[DecisionTreeRegressor] - Total Registros: 242608
[DecisionTreeRegressor] - Total Absolute Diff : 1145394
[DecisionTreeRegressor] - ME Final : 4.721171
----------------------------------------------------------
MPE - Mean Percentage Error:  
[DecisionTreeRegressor] - Total Registros: 242608
[DecisionTreeRegressor] - Total percent Diff: -705116
[DecisionTreeRegressor] - MPE Final : -2.91%
--------------------------------------------------

# Prioridade 4 - FinanciavelAtualFinal

In [18]:
import pandas as pd
consulta = cdt.resultQuery("mdl_historicoscorrentes.sql").format(dia=data_vencimento)
base = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="mdl_historicoscorrentes_"+infoDataBases["base"],spark=False,objSpark=None)
base = base.fillna(0)
base['dtvencimento'] = pd.to_datetime(base.DataVencimentoPadrao, format="%d/%m/%Y")

base = base.loc[base.dtvencimento.dt.day == data_vencimento]

mylist = ["SaldoExtratoAnterior","FinanciavelExtrato","SaldoAtualFinal","CreditosNacionais","DebitosNacionais","SaquesNacionais","Multa","FinanciavelAtualFinal","NaoFinanciavelAtualFinal","BaseMultaCobrada"]
zeros = base[base[mylist].eq(0).all(1)]

final = base.drop(zeros.index, axis=0)

treino = final.drop(final.loc[base.dtvencimento.dt.month == 4].loc[base.dtvencimento.dt.year == 2020].index,axis = 0)
teste = final.loc[base.dtvencimento.dt.month == 4].loc[base.dtvencimento.dt.year == 2020]

## Pré-Processamento - FinanciavelAtualFinal

In [19]:
from sklearn.preprocessing import StandardScaler
import time
start_time = time.time()

X = treino.drop(['FinanciavelAtualFinal','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
Y = treino['FinanciavelAtualFinal'].to_frame()

scalerX = StandardScaler()
scalerX.fit(X)
Xss = scalerX.transform(X)

scalerY = StandardScaler()
scalerY.fit(Y)
Yss = scalerY.transform(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xss, Yss, test_size = 0.3,random_state = 0)

print(str(time.time() - start_time))

0.49901676177978516


## Decision Tree - Financiavel Atual Final

In [20]:
from sklearn import tree
dt = tree.DecisionTreeRegressor()
dt = dt.fit(X_train, y_train)
mdl_financiavelatualfinal = dt

## Validate - FinanciavelAtualFinal

In [21]:
realY = teste['FinanciavelAtualFinal'].values
realX =  teste.drop(['FinanciavelAtualFinal','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
realXS = scalerX.transform(realX)

preditoYS = dt.predict(realXS)
preditoY = scalerY.inverse_transform(preditoYS)

df_dict = {'Real':realY.astype(int),'Previsto': preditoY.astype(int)}

rp = pd.DataFrame(df_dict)

a = preditoY.astype(int).reshape(preditoY.astype(int).shape[0],1)
b = realY.astype(int).reshape(realY.astype(int).shape[0],1)

a = preditoY.reshape(preditoY.shape[0],1)
df_final = cdt.transformDataSet(a, b)
cdt.getresultdataset(df_final, "DecisionTreeRegressor")

SE - Sum of Error: 
[DecisionTreeRegressor] - Total Real:  721620464.00
[DecisionTreeRegressor] - Total Prev:  721425285.34
[DecisionTreeRegressor] - SE Final:  -195179.41
[DecisionTreeRegressor] - Total Porcentagem de diferença do previsto x real:  0.03%
----------------------------------------------------------
ME - Mean Error:  
[DecisionTreeRegressor] - Total Registros: 243039
[DecisionTreeRegressor] - Total Diff : -195179
[DecisionTreeRegressor] - ME Final : -0.803079
----------------------------------------------------------
MAE - Mean Absolute Error:  
[DecisionTreeRegressor] - Total Registros: 243039
[DecisionTreeRegressor] - Total Absolute Diff : 10611766
[DecisionTreeRegressor] - ME Final : 43.662812
----------------------------------------------------------
MPE - Mean Percentage Error:  
[DecisionTreeRegressor] - Total Registros: 243039
[DecisionTreeRegressor] - Total percent Diff: -181262
[DecisionTreeRegressor] - MPE Final : -0.75%
-----------------------------------------

## Pré-Processamento - FinanciavelExtrato

In [23]:
from sklearn.preprocessing import StandardScaler
import time
start_time = time.time()

X = treino.drop(['FinanciavelExtrato','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
Y = treino['FinanciavelExtrato'].to_frame()

scalerX = StandardScaler()
scalerX.fit(X)
Xss = scalerX.transform(X)

scalerY = StandardScaler()
scalerY.fit(Y)
Yss = scalerY.transform(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xss, Yss, test_size = 0.3,random_state = 0)

print(str(time.time() - start_time))

0.4920341968536377


## Decision Tree - FinanciavelExtrato

In [24]:
from sklearn import tree
dt = tree.DecisionTreeRegressor()
dt = dt.fit(X_train, y_train)
mdl_financiavelextrato = dt

## Validate - FinanciavelExtrato

In [25]:
realY = teste['FinanciavelExtrato'].values
realX =  teste.drop(['FinanciavelExtrato','Id_Conta','DataVencimentoPadrao','dtvencimento'], axis=1)
realXS = scalerX.transform(realX)

preditoYS = dt.predict(realXS)
preditoY = scalerY.inverse_transform(preditoYS)

df_dict = {'Real':realY.astype(int),'Previsto': preditoY.astype(int)}

rp = pd.DataFrame(df_dict)

a = preditoY.astype(int).reshape(preditoY.astype(int).shape[0],1)
b = realY.astype(int).reshape(realY.astype(int).shape[0],1)

a = preditoY.reshape(preditoY.shape[0],1)
df_final = cdt.transformDataSet(a, b)
cdt.getresultdataset(df_final, "DecisionTreeRegressor")

SE - Sum of Error: 
[DecisionTreeRegressor] - Total Real:  715865974.00
[DecisionTreeRegressor] - Total Prev:  715743372.27
[DecisionTreeRegressor] - SE Final:  -122603.41
[DecisionTreeRegressor] - Total Porcentagem de diferença do previsto x real:  0.02%
----------------------------------------------------------
ME - Mean Error:  
[DecisionTreeRegressor] - Total Registros: 243417
[DecisionTreeRegressor] - Total Diff : -122603
[DecisionTreeRegressor] - ME Final : -0.503676
----------------------------------------------------------
MAE - Mean Absolute Error:  
[DecisionTreeRegressor] - Total Registros: 243417
[DecisionTreeRegressor] - Total Absolute Diff : 12272420
[DecisionTreeRegressor] - ME Final : 50.417266
----------------------------------------------------------
MPE - Mean Percentage Error:  
[DecisionTreeRegressor] - Total Registros: 243417
[DecisionTreeRegressor] - Total percent Diff: -2647332
[DecisionTreeRegressor] - MPE Final : -10.88%
---------------------------------------

# Validando com ultimo vencimento faturado

In [22]:
#consulta = cdt.resultQuery("historicoscorrentes.sql").format(dia=data_vencimento)
#base = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="historicoscorrentes_"+infoDataBases["base"],spark=False,objSpark=None)